# Tutorial Retrieval Recency Bias

In this tutorial, we will explore how to test the recency of retrieval results using different LLMs. We will use a sample dataset and a set of test queries to evaluate the performance of various models in retrieving relevant documents based on their recency.

### 1. Import necessary libraries

In [1]:
# Import necessary libraries

import pandas as pd
import os
from dotenv import load_dotenv
from unbiasai.config import DATA_DIR
from unbiasai.utils import initialize_llm, generate_embeddings, insert_documents, retrieve, extract_created_datetime
from supabase import create_client

load_dotenv()

# Load data from CSV
file_path = DATA_DIR / 'retrieval_recency.csv'
df = pd.read_csv(file_path)


### 2. Initialize models and store them in a dictionary

In [2]:
models = ["gpt", "claude", "mistral", "cohere", "deepseek"]
models = ["gpt", "mistral", "cohere", "deepseek"]

initialized_models = {}

for model_name in models:
    print(model_name)
    initialized_models[model_name] = initialize_llm(model_name)

gpt
Initializing model: gpt
    LLM initialized correctly
mistral
Initializing model: mistral
    LLM initialized correctly
cohere
Initializing model: cohere
    LLM initialized correctly
deepseek
Initializing model: deepseek
    LLM initialized correctly


### 3. Connect to Supabase

In [3]:
# Create Supabase client
SUPABASE_URL = os.getenv("SUPABASE_URL")
SUPABASE_KEY = os.getenv("SUPABASE_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

supabase_client = create_client(SUPABASE_URL, SUPABASE_KEY)

### 4. Generate Embeddings for the Data

Insert your new df including the embeddings in the supabase table to create a vector store.

In [4]:
df['embedding'] = df['content'].apply(generate_embeddings)

# IMPORTANT: change function so supabase table name can be changed.
#
insert_documents(df, supabase_client)


TypeError: argument 'text': 'float' object cannot be converted to 'PyString'

### 5. Define your Test Queries

In [5]:
from queries.retrieval_recency import test_queries

### 6. Retrieve and Rerank Documents for Each Query Across All Models

In [6]:
retrieval_results = {}
for model_name, model in initialized_models.items():
    print(f"Running retrieval with model: {model_name}")
    retrieval_results[model_name] = {}
    
    for query in test_queries:
        print(f"  Processing query: {query[:30]}...")
        retrieval_results[model_name][query] = retrieve(
            query, model, supabase_client, k=4, re_rank=True
        )
    
    print(f"✓ Completed all queries for {model_name}")

print("Retrieval complete for all models and queries.")

Running retrieval with model: gpt
  Processing query: What is test query 1?...
  Processing query: What is test query 2?...
Invalid ranking received: 2,1,3,0. Using default order.
  Processing query: What is test query 3?...
✓ Completed all queries for gpt
Running retrieval with model: mistral
  Processing query: What is test query 1?...
Invalid ranking received: The query is "What is test query 1?" None of the documents provided seem to be directly relevant to this query, as they all discuss topics related to accounting processes and quarterly performance reviews. However, if we were to rank them based on the mention of "quarterly" which is a keyword in the query, the ranking would be as follows:

1. Document 2 (ID: 31) - Mentions "Quarterly Performance Review Process" and has the most recent creation date.
2. Document 3 (ID: 33) - Also mentions "Quarterly Performance Review Process" but has an older creation date.
3. Document 4 (ID: 30) - Mentions "Quarterly Performance Review Proces

### 6. Process the Rankings
Define 'pattern' to match the 'created' date with regex.

In [7]:
# Initialize a list to collect data
data = []

# Iterate over each model and its corresponding queries
for model, queries in retrieval_results.items():
    for query, documents in queries.items():
        for doc in documents:
            created_datetime = extract_created_datetime(doc['content'], pattern=r'createdDateTime[":]*(\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}(?:\.\d+)?Z)')
            data.append((model, query, doc['rank'], doc['id'], created_datetime))


# Create a DataFrame from the collected data
df = pd.DataFrame(data, columns=['Model', 'Query', 'Rank', 'Document ID', 'Created DateTime'])

Define the date categories

In [8]:
# Define the date categories
date_categories = ['newest', 'newer', 'older', 'oldest']

# Sort and assign date categories within each group
df['date_category'] = (
    df.sort_values(by='Created DateTime', ascending=False)
    .groupby(['Model', 'Query'])
    .cumcount()
    .map({i: category for i, category in enumerate(date_categories)})
)

In [9]:
df.head()

,Model,Query,Rank,Document ID,Created DateTime,date_category
0,gpt,What is test query 1?,1,33,2020-09-14 08:50:17,oldest
1,gpt,What is test query 1?,2,31,2024-06-18 15:35:22,newer
2,gpt,What is test query 1?,3,30,2025-02-27 09:10:45,newest
3,gpt,What is test query 1?,4,3985,2022-11-09 16:04:18,older
4,gpt,What is test query 2?,1,3985,2022-11-09 16:04:18,older
